In [1]:
# Import
import pandas as pd
from lxml import html
from bs4 import BeautifulSoup as bs
from IPython.display import clear_output
import numpy as np

In [2]:
# Variables
file_name = "FFS Spreadsheet.xlsx"

#### OPTIONAL FOR TEAM SPECIFIC REPORTS ####
#ffs_teams_list = ['CIN', 'BAL']
ffs_teams_list = ['ARI','ATL','BAL','BOS','CHC','CIN','CLE','COL','CHW','DET','HOU','KC','LAA','LAD','MIA','MIL','MIN','NYM','NYY','OAK','PHI','PIT','SD','SEA','SF','STL','TB','TEX','TOR','WAS']


In [3]:
# Pick number function

def get_pick_info(pick, current_year):
    
    cy2 = f'{current_year[-2]}{current_year[-1]}'
    if cy2 in pick:
    
        # Draft Order
        draft_order = ({
            "teams": ffs_teams_list,
            "pick_number": [27,21,28,20,24,6,11,12,1,30,22,25,18,4,9,29,14,19,17,8,3,16,15,2,23,13,10,26,7,5]
        })
    
        # Rounds
        pick_num = 0
        
        if '1ST' in pick:
            pick_round = '1'
        #if '1A' in pick:
        #    pick_num += 30
        #    pick_round = '1A'
        if '2ND' in pick:
            pick_num += 30
            pick_round = '2'
        if '3RD' in pick:
            pick_num += 60
            pick_round = '3'
        #if '3A' in pick:
        #    pick_num += 96
        #    pick_round = '3A'
        if '4TH' in pick:
            pick_num += 90
            pick_round = '4'
        if '5TH' in pick:
            pick_num += 120
            pick_round = '5'
        if '6TH' in pick:
            pick_num += 150
            pick_round = '6'
        if '7TH' in pick or '8TH' in pick or '9TH' in pick or '10TH' in pick or '11TH' in pick:
            return
    
        # Sandwich Picks
        if 'P1' in pick:
            pick_num += 1
            round_pick = '1'
        if 'P2' in pick:
            pick_num += 2
            round_pick = '2'
        if 'P3' in pick:
            pick_num += 3
            round_pick = '3'
        if 'P4' in pick:
            pick_num += 4
            round_pick = '4'
        if 'P5' in pick:
            pick_num += 5
            round_pick = '5'
        if 'P6' in pick:
            pick_num += 6
            round_pick = '6'
        
        # Get draft order pick number
        pick_count = 0
        for team in draft_order['teams']:
            
            # Get pick of the round
            if team in pick:
                pick_num += draft_order['pick_number'][pick_count]
                round_pick = draft_order['pick_number'][pick_count]
            pick_count += 1
        
        
        return pick_num, pick_round, round_pick

In [4]:
# Import FFS MLB Sheet
# Read Player Names
cols = ['S','U','W']
current_year = "2024"
year_minus1 = str(int(current_year) - 1)
year_plus1 = str(int(current_year) + 1)
year_plus2 = str(int(current_year) + 2)

draft_sheet = []

# Loop through each tab to pull list of player names
for team in ffs_teams_list:
    
    # Write progress to console
    print(f"Processing data for {team}") 
        
    for col in cols:
        # Variables
        sheet = pd.read_excel(file_name, sheet_name=team, usecols=col, skiprows=1, index_col=None, header=0)
        # Remove empty rows
        nan_value = float("NaN")
        sheet.replace("", nan_value, inplace=True)
        sheet.dropna(inplace=True)
    
        picks = sheet.values
        for pick in picks:
            try:
                pick_num, pick_round, round_pick = get_pick_info(pick[0], current_year)
            except:
                pick_num = None
            if pick_num != None:
                team_dict = {
                    "round_and_pick": f'{pick_round}.{round_pick} ({pick_num})',
                    "Due Date": "",
                    "pick": pick[0],
                    "team": team,
                    "pick_num": pick_num,
                    "Selection": ""
                }
                draft_sheet.append(team_dict)
        
    # Clear console output
    #clear_output(wait=True)
    

Processing data for ARI
Processing data for ATL
Processing data for BAL
Processing data for BOS
Processing data for CHC
Processing data for CIN
Processing data for CLE
Processing data for COL
Processing data for CHW
Processing data for DET
Processing data for HOU
Processing data for KC
Processing data for LAA
Processing data for LAD
Processing data for MIA
Processing data for MIL
Processing data for MIN
Processing data for NYM
Processing data for NYY
Processing data for OAK
Processing data for PHI
Processing data for PIT
Processing data for SD
Processing data for SEA
Processing data for SF
Processing data for STL
Processing data for TB
Processing data for TEX
Processing data for TOR
Processing data for WAS


In [5]:
# Show results
draft_sheet_df = pd.DataFrame(draft_sheet)
draft_sheet_df = draft_sheet_df.sort_values(by=['pick_num'])
draft_sheet_df

,round_and_pick,Due Date,pick,team,pick_num,Selection
39,1.1 (1),,1ST CHW 24,CHW,1,
0,1.2 (2),,1ST SEA 24,ARI,2,
123,1.3 (3),,1ST PHI 24,PHI,3,
73,1.4 (4),,1ST LAD 24,LAD,4,
177,1.5 (5),,1ST WAS 24,WAS,5,
...,...,...,...,...,...,...
22,6.26 (176),,6TH TEX 24,CHC,176,
4,6.27 (177),,6TH ARI 24,ARI,177,
16,6.28 (178),,6TH BAL 24,BAL,178,
55,6.29 (179),,6TH MIL 24,CHW,179,


In [6]:
# Export csv
draft_sheet_df.set_index(['pick_num'], inplace = True)
#draft_sheet_df = draft_sheet_df.drop(['pick_num'], axis=1)
draft_sheet_df.to_csv(f"draft/draft_order.csv", index = False)
draft_sheet_df

,round_and_pick,Due Date,pick,team,Selection
pick_num,,,,,
1,1.1 (1),,1ST CHW 24,CHW,
2,1.2 (2),,1ST SEA 24,ARI,
3,1.3 (3),,1ST PHI 24,PHI,
4,1.4 (4),,1ST LAD 24,LAD,
5,1.5 (5),,1ST WAS 24,WAS,
...,...,...,...,...,...
176,6.26 (176),,6TH TEX 24,CHC,
177,6.27 (177),,6TH ARI 24,ARI,
178,6.28 (178),,6TH BAL 24,BAL,


In [7]:
# Export HTML for Proboards
html = draft_sheet_df.to_html(index=False)
html = html.replace('<', '[').replace('>', ']').replace('[td', '[td style="border:1px solid rgb(0, 0, 0);padding:3px;width:120px"').replace('[table border="1" class="dataframe"]', '[table]').replace('team', 'Owner').replace('pick', 'Pick Name').replace('round_and_Pick Name', 'Round.Pick (Overall)').replace('[tr style="text-align: right;"]', '[tr style="text-align: center;"]').replace('[tr]', '[tr style="text-align: center;"]')
print(html)

[table]
  [thead]
    [tr style="text-align: center;"]
      [th]Round.Pick (Overall)[/th]
      [th]Due Date[/th]
      [th]Pick Name[/th]
      [th]Owner[/th]
      [th]Selection[/th]
    [/tr]
  [/thead]
  [tbody]
    [tr style="text-align: center;"]
      [td style="border:1px solid rgb(0, 0, 0);padding:3px;width:120px"]1.1 (1)[/td]
      [td style="border:1px solid rgb(0, 0, 0);padding:3px;width:120px"][/td]
      [td style="border:1px solid rgb(0, 0, 0);padding:3px;width:120px"]1ST CHW 24[/td]
      [td style="border:1px solid rgb(0, 0, 0);padding:3px;width:120px"]CHW[/td]
      [td style="border:1px solid rgb(0, 0, 0);padding:3px;width:120px"][/td]
    [/tr]
    [tr style="text-align: center;"]
      [td style="border:1px solid rgb(0, 0, 0);padding:3px;width:120px"]1.2 (2)[/td]
      [td style="border:1px solid rgb(0, 0, 0);padding:3px;width:120px"][/td]
      [td style="border:1px solid rgb(0, 0, 0);padding:3px;width:120px"]1ST SEA 24[/td]
      [td style="border:1px solid rgb(

,round_and_pick,Due Date,pick,team,Selection
pick_num,,,,,
1,1.1 (1),,1ST CHW 24,CHW,
2,1.2 (2),,1ST SEA 24,ARI,
3,1.3 (3),,1ST PHI 24,PHI,
4,1.4 (4),,1ST LAD 24,LAD,
5,1.5 (5),,1ST WAS 24,WAS,
...,...,...,...,...,...
176,6.26 (176),,6TH TEX 24,CHC,
177,6.27 (177),,6TH ARI 24,ARI,
178,6.28 (178),,6TH BAL 24,BAL,
